In [ ]:
# Install required packages
import subprocess
import sys

packages = ['SpeechRecognition', 'ollama', 'pyttsx3', 'pyaudio']

for package in packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")
    except Exception as e:
        print(f"✗ Error installing {package}: {e}")

In [ ]:
import speech_recognition as sr
import ollama
import pyttsx3

## The Ears - Speech to Text Function

This function captures audio from the microphone and converts it to text using Google's Web Speech API.

In [ ]:
def listen():
    """Capture audio from microphone and convert to text"""
    recognizer = sr.Recognizer()
    
    try:
        with sr.Microphone() as source:
            print("Listening... (Speak now)")
            # Adjust for ambient noise
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            
            # Listen for audio input
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
            print("Processing...")
        
        # Recognize speech using Google's free API
        text = recognizer.recognize_google(audio)
        print(f"You said: {text}")
        return text
    
    except sr.WaitTimeoutError:
        print("No speech detected (timeout).")
        return None
    except sr.UnknownValueError:
        print("Could not understand audio.")
        return None
    except Exception as e:
        print(f"An error occurred in listen(): {e}")
        return None

## The Brain - LLM Inference Function

This function sends the user's text to the Llama 3 model running locally via Ollama for intelligent response generation.

In [ ]:
def think(text: str):
    """Process text through Llama 3 model locally via Ollama"""
    if not text:
        return None
    
    print("Thinking...")
    
    try:
        # Ensure you have pulled the model via: ollama pull llama3
        response = ollama.chat(
            model="llama3",
            messages=[
                {
                    "role": "user",
                    "content": text,
                }
            ],
        )
        
        response_text = response["message"]["content"]
        print(f"AI: {response_text}")
        return response_text
    
    except Exception as e:
        print(f"An error occurred in think(): {e}")
        return "Sorry, something went wrong while thinking."

## The Mouth - Text to Speech Function

This function converts the AI's text response back to audio so the user can hear the response.

In [ ]:
def speak(text: str):
    """Convert text to speech using pyttsx3"""
    if not text:
        return
    
    try:
        engine = pyttsx3.init()
        
        # Optional: Change voice properties
        voices = engine.getProperty("voices")
        if voices:
            # Try changing index 0 -> 1 for alternative voice
            engine.setProperty("voice", voices[0].id)
        
        engine.setProperty("rate", 175)  # Speed of speech
        
        engine.say(text)
        engine.runAndWait()
    
    except Exception as e:
        print(f"An error occurred in speak(): {e}")

## The Main Function

This orchestrates the complete loop: Listen -> Think -> Speak, and repeat until the user says "exit", "stop", or "quit".

In [ ]:
def main():
    """Main loop for the voice AI assistant"""
    print("--- Voice Assistant Started ---")
    speak("Hello, I am ready. You can start speaking.")
    
    while True:
        # 1. Listen
        user_input = listen()
        
        # Skip if nothing heard
        if not user_input:
            continue
        
        # 2. Check for exit keywords
        if user_input.lower().strip() in ["exit", "stop", "quit"]:
            speak("Goodbye!")
            print("Exiting...")
            break
        
        # 3. Think
        ai_response = think(user_input)
        
        # 4. Speak
        speak(ai_response)

## Run the Voice AI Assistant

⚠️ **Note**: Running this cell will start the assistant. It will:
- Listen for your voice input
- Think using the local Llama 3 model
- Speak back the response

Say "exit", "stop", or "quit" to end the session.

**Important**: Make sure:
1. Ollama is running on your machine
2. You have pulled the Llama 3 model (`ollama pull llama3`)
3. Your microphone and speakers are working

In [ ]:

main()

print("To run the assistant, uncomment the main() call above and execute this cell.")
print("The assistant will listen for your voice and respond intelligently.")

## Troubleshooting

### Ollama not installed
Download from: https://ollama.ai/

### Llama 3 model not pulled
Run in your terminal: `ollama pull llama3`

### PyAudio issues on Windows
You may need to install Visual C++ Build Tools or use: `pip install pipwin` then `pipwin install pyaudio`

### Microphone not working
Check that your microphone is properly connected and enabled in system settings.

### Google Speech Recognition not working
This requires an internet connection. Ensure your network is connected.